<a href="https://colab.research.google.com/github/cboyda/LighthouseLabs/blob/main/Project_Stats_yelp_foursquare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests # library we use for sending various requests from Python is called requests
import os # used to access the values of environment variables
import json
import pandas as pd

In [5]:
# how to mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Grab city bikes API information to find locations to parse:

In [47]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/Project-Statistics_city_bikes.csv')  

# Foursquare

In [3]:
api_key = os.environ["FOURSQUARE_API_KEY"]
# os.environ loads the dictionary with environment variables where os.environ.keys() are all variable names

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [65]:
def get_venues(place):
  location = place
  url = "https://api.foursquare.com/v3/places/search?near=" + location
  # Create dictionary for headers
  headers = {"Accept": "application/json"}
  # Add key with our API KEY
  headers['Authorization'] = api_key

  try:
    response = requests.get(url, headers=headers) # result should return '<Response [200]>'

    # Extract the JSON content from the response
    json_data = response.json()

    # Access the 'results' key in the JSON response
    results = json_data['results']

    bar_counter = 0
    for result in results:
        categories = result['categories']
        for category in categories:
            # if 'Bar' in category['name']:
            #     bar_counter += 1
            if (category['id'] == 16032) or (category['id'] == 16033) or (category['id'] == 16034) or (category['id'] == 16035) or (category['id'] == 16036) or (category['id'] == 16037) or (category['id'] == 16038):
                bar_counter += 1
    # if bar_counter > 0:
      # print(location,"has", bar_counter,"bars in the area") # for debugging
    return bar_counter
  except requests.exceptions.RequestException as e:
    print("Error:",e)
    # Get the value of 'X-RateLimit-Reset' from the response headers
    rate_limit_reset = response.headers.get('X-RateLimit-Reset')
    print(rate_limit_reset)  
    return -99999

Let's look for nearby PARKS, FourSquare API categoryid:
* 16032	Landmarks and Outdoors > Park
* 16033	Landmarks and Outdoors > Park > Dog Park
* 16034	Landmarks and Outdoors > Park > National Park
* 16035	Landmarks and Outdoors > Park > Natural Park
* 16036	Landmarks and Outdoors > Park > Picnic Area
* 16037	Landmarks and Outdoors > Park > Playground
* 16038	Landmarks and Outdoors > Park > State or Provincial Park
* 16039	Landmarks and Outdoors > Park > Urban Park

Can not just use "park" in Category Name, since then we get trailer parks etc.


In [36]:
df

,city,station_name,empty_slots,slots,free_bikes,ebikes,latitude,longitude
0,Vancouver,10th & Cambie,22,35,13,4,49.262487,-123.114397
1,Vancouver,Yaletown-Roundhouse Station,6,16,10,0,49.274566,-123.121817
2,Vancouver,Dunsmuir & Beatty,23,26,3,1,49.279764,-123.110154
3,Vancouver,12th & Yukon (City Hall),14,16,2,2,49.260599,-123.113504
4,Vancouver,8th & Ash,15,16,1,0,49.264215,-123.117772
...,...,...,...,...,...,...,...,...
237,Vancouver,Burrard & 14th,12,18,4,4,49.259469,-123.145718
238,Vancouver,Hornby & Drake,19,24,5,2,49.277178,-123.130000
239,Vancouver,Cardero & Bayshore,2,20,16,1,49.291597,-123.129158
240,Vancouver,27th & Main,13,22,9,7,49.247204,-123.101549


In [42]:
# total number of rows?
df.shape[0]

242

In [38]:
# are these locations unique?
df[:].nunique()

city              1
station_name    242
empty_slots      26
slots            20
free_bikes       29
ebikes            9
latitude        242
longitude       241
dtype: int64

In [59]:
# create location column with lat, long
#df['station_location'] = df['latitude'] + "," + df['longitude']
df['station_location'] = df['latitude'].astype(str) + "," + df['longitude'].astype(str)

# initiatlize location_count which will return how many are nearby
df['location_count'] = -1

In [48]:
unique_locations = df['station_location'].tolist()
print("The length of unique_locations remaining is",len(unique_locations))

The length of unique_locations remaining is 242


In [60]:
# loop through each station location in data to update location_count 
for location in unique_locations:
    rows_to_update = df['station_location'] == location
    if (df.loc[rows_to_update, 'location_count'] == -1).any():
        # Set 'bar_count' to the value returned by 'get_venues' function
        # data.loc[rows_to_update, 'bar_count'] = get_venues(location) # needs error handling
        location_count = get_venues(location)
        if location_count == -99999:
            break # stop making API calls
        else:
            df.loc[rows_to_update, 'location_count'] = location_count
    # save our API calls and work to a hard copy file
    # data.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/W5D1monroe-county-crash-data2003-to-2015_barcounts.csv', index=False)  # Specify the desired file path


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [61]:
df

,city,station_name,empty_slots,slots,free_bikes,ebikes,latitude,longitude,station_location,location_count
0,Vancouver,10th & Cambie,22,35,13,4,49.262487,-123.114397,"49.262487,-123.114397",0
1,Vancouver,Yaletown-Roundhouse Station,6,16,10,0,49.274566,-123.121817,"49.274566,-123.121817",0
2,Vancouver,Dunsmuir & Beatty,23,26,3,1,49.279764,-123.110154,"49.279764,-123.110154",0
3,Vancouver,12th & Yukon (City Hall),14,16,2,2,49.260599,-123.113504,"49.260599,-123.113504",0
4,Vancouver,8th & Ash,15,16,1,0,49.264215,-123.117772,"49.264215,-123.117772",0
...,...,...,...,...,...,...,...,...,...,...
237,Vancouver,Burrard & 14th,12,18,4,4,49.259469,-123.145718,"49.259469,-123.145718",0
238,Vancouver,Hornby & Drake,19,24,5,2,49.277178,-123.130000,"49.277178,-123.13",0
239,Vancouver,Cardero & Bayshore,2,20,16,1,49.291597,-123.129158,"49.291597,-123.129158",0
240,Vancouver,27th & Main,13,22,9,7,49.247204,-123.101549,"49.247204,-123.101549",0


Put your parsed results into a DataFrame

In [64]:
df[df["location_count"] > 0]

,city,station_name,empty_slots,slots,free_bikes,ebikes,latitude,longitude,station_location,location_count
5,Vancouver,Spyglass & Seawall,2,14,12,0,49.270877,-123.115103,"49.270877,-123.115103",3
6,Vancouver,Stamps Landing,7,14,7,1,49.268930,-123.118130,"49.26893,-123.11813",4
7,Vancouver,Ontario & Seawall,0,20,20,0,49.271721,-123.104207,"49.271721,-123.104207",1
11,Vancouver,Marinaside & Davie,12,22,10,0,49.273471,-123.118016,"49.273471,-123.118016",2
19,Vancouver,Cordova & Bute,9,14,5,0,49.289124,-123.121960,"49.289124,-123.12196",1
...,...,...,...,...,...,...,...,...,...,...
227,Vancouver,30th & Ontario,10,22,12,5,49.243375,-123.104978,"49.243375,-123.104978",1
228,Vancouver,Franklin & Nanaimo,11,20,9,2,49.282087,-123.056909,"49.282087,-123.056909",1
229,Vancouver,Lakewood & Triumph,14,16,2,0,49.283968,-123.061615,"49.283968,-123.061615",1
230,Vancouver,Wall & Eton,4,18,14,0,49.287464,-123.060771,"49.287464,-123.060771",2


In this example the "location_count" returns how many parks are nearby.

In [66]:
# save our API calls and work to a hard copy file
# df.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/Project-Statistics_city_bikes_FourSquare.csv', index=False)  # Specify the desired file path


In [107]:
df_foursquare = df.copy()

# Yelp

In [17]:
# manual experimentation
api_key = os.environ["FOURSQUARE_API_KEY"]
# os.environ loads the dictionary with environment variables where os.environ.keys() are all variable names


## Manual Yelp API Experimentation

In [18]:
# test simple API call for validty
debug = False

if debug:
  import requests

  url = "https://api.yelp.com/v3/businesses/search?latitude=49.270877&longitude=-123.115103&term=park&radius=1000&sort_by=best_match&limit=20"

  headers = {
      "accept": "application/json",
      "Authorization": yelp_api_key
  }

  response = requests.get(url, headers=headers)

  print(response.text)

In [3]:
# response.text

'{"businesses": [{"id": "-fHqwIU4S-uD9O-ouzhE_A", "alias": "rainbow-park-vancouver", "name": "Rainbow Park", "image_url": "https://s3-media4.fl.yelpcdn.com/bphoto/5sMzPag0gWwoNyYrwhjnig/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/rainbow-park-vancouver?adjust_creative=yFWeVz8UGIGNocCyw6wXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=yFWeVz8UGIGNocCyw6wXdw", "review_count": 3, "categories": [{"alias": "parks", "title": "Parks"}], "rating": 5.0, "coordinates": {"latitude": 49.27917223142565, "longitude": -123.11873478406143}, "transactions": [], "location": {"address1": "872 Richards Street", "address2": "", "address3": null, "city": "Vancouver", "zip_code": "V6B 3B4", "country": "CA", "state": "BC", "display_address": ["872 Richards Street", "Vancouver, BC V6B 3B4", "Canada"]}, "phone": "", "display_phone": "", "distance": 959.278686538304}, {"id": "lqhQMVcOu6NDNIQKP7a0rg", "alias": "david-lam-park-vancouver", "name": "David Lam Park", "image_

In [98]:
debug = True
if debug:
  # parse output from JSON call, and output nicely human-readable format

  # Assume data contains the response text
  data = response.text

  # Parse the response text as JSON
  data = json.loads(data)

  # Access the businesses list
  businesses = data["businesses"]

  # Iterate over each business
  for business in businesses:
      # Access the business details
      business_id = business["id"]
      alias = business["alias"]
      name = business["name"]
      image_url = business["image_url"]
      is_closed = business["is_closed"]
      url = business["url"]
      review_count = business["review_count"]
      categories = business["categories"]
      rating = business["rating"]

      # careful this is ANOTHER API call and we only have max 500 per day
      # if we wanted to see the details of each individual review
      countReviews = False
      if countReviews:
        # Set up the API endpoint and parameters for fetching reviews
        reviews_url = f"https://api.yelp.com/v3/businesses/{business_id}/reviews"
        reviews_response = requests.get(reviews_url, headers=headers)
        reviews_data = json.loads(reviews_response.text)
        reviews = reviews_data["reviews"]
      
        # Access the review score
        if reviews:
            review_score = reviews[0]["rating"]
        else:
            review_score = None


      # Display business details
      print("Business ID:", business_id)
      print("Alias:", alias)
      print("Name:", name)
      print("Image URL:", image_url)
      print("Is Closed:", is_closed)
      print("URL:", url)
      print("Rating:", rating)
      print("Review Count:", review_count)
      if countReviews:
        print("Review Score:", review_score)
      print("Categories:", categories)
      print()

# SAMPLE OUTPUT for each business:
# Business ID: -fHqwIU4S-uD9O-ouzhE_A
# Alias: rainbow-park-vancouver
# Name: Rainbow Park
# Image URL: https://s3-media4.fl.yelpcdn.com/bphoto/5sMzPag0gWwoNyYrwhjnig/o.jpg
# Is Closed: False
# URL: https://www.yelp.com/biz/rainbow-park-vancouver?adjust_creative=yFWeVz8UGIGNocCyw6wXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=yFWeVz8UGIGNocCyw6wXdw
# Review Count: 3
# Categories: [{'alias': 'parks', 'title': 'Parks'}]

Business ID: -fHqwIU4S-uD9O-ouzhE_A
Alias: rainbow-park-vancouver
Name: Rainbow Park
Image URL: https://s3-media4.fl.yelpcdn.com/bphoto/5sMzPag0gWwoNyYrwhjnig/o.jpg
Is Closed: False
URL: https://www.yelp.com/biz/rainbow-park-vancouver?adjust_creative=yFWeVz8UGIGNocCyw6wXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=yFWeVz8UGIGNocCyw6wXdw
Rating: 5.0
Review Count: 3
Categories: [{'alias': 'parks', 'title': 'Parks'}]

Business ID: lqhQMVcOu6NDNIQKP7a0rg
Alias: david-lam-park-vancouver
Name: David Lam Park
Image URL: https://s3-media1.fl.yelpcdn.com/bphoto/qmd0FsSQjR3hZPP-8ncMfg/o.jpg
Is Closed: False
URL: https://www.yelp.com/biz/david-lam-park-vancouver?adjust_creative=yFWeVz8UGIGNocCyw6wXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=yFWeVz8UGIGNocCyw6wXdw
Rating: 4.5
Review Count: 18
Categories: [{'alias': 'parks', 'title': 'Parks'}]

Business ID: nGAwccdDf9rFW4GTVpH2vw
Alias: emery-barnes-park-vancouver
Name: Emery Barnes Park
Imag

## Yelp API Functions

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [77]:
def get_yelp_venues(latitude,longitude):
  '''
  Receives latitude,longitude as floats information to lookup in Yelp API
  Looksup any nearby parks or related, with maximum 20 listings.
  RETURNS: [a,b,c] were both each are integers, 
    a is the count of parks 
    b is the total review counts
    c is yelp rate limit remaining
  '''
  # web URL https://www.yelp.com/search?find_desc=Parks+Near+Me&find_loc=Edmonton%2C+AB%2C+Canada
  # bars example url = "https://api.yelp.com/v3/businesses/search?location=" 
  # + location + "&categories=bars&open_now=true$limit=40&offset=80"
  url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(latitude) + "&longitude=" + str(longitude) + "&term=park&radius=1000&sort_by=best_match&limit=20"

  # Create dictionary for headers
  headers = {"Accept": "application/json"}
  # Add key with our API KEY
  headers['Authorization'] = yelp_api_key

  try:
    response = requests.get(url, headers=headers) # result should return '<Response [200]>'

    # Extract the JSON content from the response
    json_data = response.json()
 
    # Access the businesses list
    businesses = json_data["businesses"]

    location_count = 0
    review_count = 0
    # Iterate over each business
    for business in businesses:
      categories = business['categories']
      # Check if the business category contains the alias 'parks'
      for category in categories:
          if category["alias"] == "parks": 
              review_count += int(business["review_count"])
              location_count += 1
    return [location_count,review_count,response.headers.get('RateLimit-Remaining')]

  except requests.exceptions.RequestException as e:
    print("Error:",e)
    # Get the error code and description from the response JSON
    try:
        error_json = response.json()["error"]
        error_code = error_json["code"]
        error_description = error_json["description"]
        error_message = {
            "error": {
                "code": error_code,
                "description": error_description
            }
        }
        print(json.dumps(error_message, indent=4))
    except (json.JSONDecodeError, KeyError):
        print("Failed to parse error response.")
    return -99999

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [52]:
df

,city,station_name,empty_slots,slots,free_bikes,ebikes,latitude,longitude
0,Vancouver,10th & Cambie,22,35,13,4,49.262487,-123.114397
1,Vancouver,Yaletown-Roundhouse Station,6,16,10,0,49.274566,-123.121817
2,Vancouver,Dunsmuir & Beatty,23,26,3,1,49.279764,-123.110154
3,Vancouver,12th & Yukon (City Hall),14,16,2,2,49.260599,-123.113504
4,Vancouver,8th & Ash,15,16,1,0,49.264215,-123.117772
...,...,...,...,...,...,...,...,...
237,Vancouver,Burrard & 14th,12,18,4,4,49.259469,-123.145718
238,Vancouver,Hornby & Drake,19,24,5,2,49.277178,-123.130000
239,Vancouver,Cardero & Bayshore,2,20,16,1,49.291597,-123.129158
240,Vancouver,27th & Main,13,22,9,7,49.247204,-123.101549


In [73]:
# define our unique locations list to iterate through, it is the whole dataframe but this 
# extra step is included in case future results include duplicates and we want to reduce API calls
# unique_locations = df.index.tolist()
unique_locations = list(df.index.unique())
# unique_locations = list(df.index.unique()[0:2]) # for debug testing limit first 2 items only

In [80]:
len(unique_locations)

242

In [88]:
# how many of those locations still have -1 values for yelp_location_count?
df[df["yelp_location_count"] == -1].shape
# df[df["yelp_location_count"] == -1] # returns the actual full dataframe that meets that condition

(16, 10)

In [64]:
# initialize new column with -1 so we know which rows are incomplete
df['yelp_location_count'] = -1
df['yelp_review_count'] = -1

In [90]:
debug = False
# loop through each station location in data to update location_count 
for location in unique_locations:
    rows_to_update = df.index == location
    if (df.loc[rows_to_update, 'yelp_location_count'] == -1).any():
        # Set 'yelp_location_count' to the value returned by 'get_yelp_venues' function
        venue_results = get_yelp_venues(df['latitude'][location],df['longitude'][location])
        if venue_results == -99999:
            break # stop making API calls
        else:
            df.loc[rows_to_update, 'yelp_location_count'] = venue_results[0]
            df.loc[rows_to_update, 'yelp_review_count'] = venue_results[1]
            if debug:
              print("There are",venue_results[2],"Yelp API calls remaining.")

Put your parsed results into a DataFrame

In [91]:
df

,city,station_name,empty_slots,slots,free_bikes,ebikes,latitude,longitude,yelp_location_count,yelp_review_count
0,Vancouver,10th & Cambie,22,35,13,4,49.262487,-123.114397,8,42
1,Vancouver,Yaletown-Roundhouse Station,6,16,10,0,49.274566,-123.121817,14,94
2,Vancouver,Dunsmuir & Beatty,23,26,3,1,49.279764,-123.110154,10,61
3,Vancouver,12th & Yukon (City Hall),14,16,2,2,49.260599,-123.113504,10,49
4,Vancouver,8th & Ash,15,16,1,0,49.264215,-123.117772,9,69
...,...,...,...,...,...,...,...,...,...,...
237,Vancouver,Burrard & 14th,12,18,4,4,49.259469,-123.145718,8,15
238,Vancouver,Hornby & Drake,19,24,5,2,49.277178,-123.130000,13,176
239,Vancouver,Cardero & Bayshore,2,20,16,1,49.291597,-123.129158,12,1251
240,Vancouver,27th & Main,13,22,9,7,49.247204,-123.101549,6,173


In [92]:
# save our API calls and work to a hard copy file
df.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/Project-Statistics_city_bikes_yelp.csv', index=False) 


In [7]:
df_yelp = df.copy()

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Really enjoyed working with YELP, their help documentation on https://docs.developer.yelp.com/reference/v3_business_search is perfect since it allows me to see the parameters and make test calls.  I also was able to include additional information (review_count) to see how popular the locations are.  If I had more API calls available I could also call each successful Yelp location and return the actual current review_rating!


In [120]:
# quality could be number of zero values in location column
df_yelp[df_yelp['yelp_location_count'] == 0].shape[0] # Yelp 
# YELP RETURNS 0 rows

0

In [129]:
percentage = (df_yelp[df_yelp["yelp_location_count"] == 0].shape[0] / df_yelp.shape[0]) * 100
formatted_percentage = f"{percentage:.2f}%"
print(formatted_percentage, "of Yelp rows have no locations found nearby.")

0.00% of Yelp rows have no locations found nearby.


In [121]:
# quality could be number of zero values in location column
df_foursquare[df_foursquare['location_count'] == 0].shape[0] # FourSquare 
# FourSquare RETURNS 180 rows

180

In [130]:
percentage = (df_foursquare[df_foursquare["location_count"] == 0].shape[0] / df_foursquare.shape[0]) * 100
formatted_percentage = f"{percentage:.2f}%"
print(formatted_percentage, "of FourSquare rows have no locations found nearby.")

74.38% of FourSquare rows have no locations found nearby.


Get the top 10 restaurants according to their rating

This is not a valid question since I looked for parks not resturants and I do not have enough API limits to return their ratings (but the code was created for future opportunities)!

In [8]:
# I can return the top 10 stations by the number of reviews of parks (aka popularity) in their proximity
df_yelp.sort_values("yelp_review_count", ascending=False).head(10)

,city,station_name,empty_slots,slots,free_bikes,ebikes,latitude,longitude,yelp_location_count,yelp_review_count
235,Vancouver,Cardero & Robson,3,26,23,3,49.289255,-123.132677,11,1306
48,Vancouver,Chilco & Barclay,6,18,12,0,49.291909,-123.140713,11,1294
174,Vancouver,Haro & Denman,8,20,11,2,49.290587,-123.136250,10,1292
23,Vancouver,Chilco & Beach,8,14,6,0,49.289720,-123.144028,10,1292
49,Vancouver,Chilco & Robson,9,16,7,2,49.293257,-123.138563,10,1292
215,Vancouver,Nelson & Denman,10,18,7,0,49.289071,-123.138407,10,1292
239,Vancouver,Cardero & Bayshore,2,20,16,1,49.291597,-123.129158,12,1251
103,Vancouver,Stanley Park - Information Booth,14,20,6,0,49.297933,-123.130758,9,1242
53,Vancouver,Stanley Park - Vancouver Aquarium,6,16,10,1,49.299292,-123.130281,8,1228
191,Vancouver,Hastings & Broughton,17,24,7,0,49.290112,-123.127062,9,1226
